# Klassifikation mit Support Vector Machines

Wie  gesehen hast, hat die Vorhersage erfolgreicher Artikel mit BERT sehr gut funktioniert.

Allerdings ist der dazu notwendige Rechenaufwand ziemlich erheblich. Ohne Grafikkarten dauert das mehrere Stunden. Deshalb solltest du dich in solchen Fälle fragen, ob du mit einfacheren Methoden nicht auch ein ähnlich gutes Ergebnis erzielen kannst. Das ist besonders dann wichtig, wenn du das Modell noch weiter skalieren willst (und z.B. die Newsticker-Meldungen von einem Jahrzehnt betrachten willst).

Hier wirst du daher die BERT-Ergebnisse mit einer einfachen SVM vergleichen.

## Daten einladen

Um möglichst die gleichen Voraussetzungen zu schaffen, betrachtest du nur die Titel selbst:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT id, datePublished, title, commentCount FROM articles \
                    WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

## Daten für Klassifikation vorbereiten

Zunächst normalisierst du die Kommentare:

In [ ]:
df["normalizedCommentCount"] = df["commentCount"].fillna(0).map(int)
df.loc[df["normalizedCommentCount"]>500, "normalizedCommentCount"] = 500

Dann konstruierst du zwei `DataFrame`, in denen erfolgreich und nicht erfolgreiche Posts enthalten sind:

In [ ]:
df_success = df[df["normalizedCommentCount"]>50].copy()
df_success["success"] = 1

df_no_success = df[df["normalizedCommentCount"]<10].copy()
df_no_success["success"] = 0

Du berechnest die Größe des kleineren `DataFrame`:

In [ ]:
min_success = min(len(df_success), len(df_no_success))

Und erzeugst ein ausgeglichenes Trainingsset:

In [ ]:
sdf = pd.concat([df_success.sample(min_success, random_state=42),
                 df_no_success.sample(min_success, random_state=42)])

## Vektorisieren und trainieren

Die Daten vektorisierst du nun:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,2))
tfidf_vectors = tfidf_vectorizer.fit_transform(sdf["title"])
tfidf_vectors

Um die Performance messen zu können, teilst du die Daten wie gewohnt in ein Trainings- und Testdatenset auf:

In [ ]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(tfidf_vectors, sdf["success"].values, 
                                                      train_size=0.75, random_state=42,
                                                      stratify=sdf["success"].values)

Schließlich trainierst du das Modell:

In [ ]:
from sklearn.svm import SVC
svc = SVC(random_state=42)
svc.fit(X_train, y_train)

## Vorhersagen

Für die Testdaten lässt du die Vorhersagen berechnen:

In [ ]:
import numpy as np
pred_test  = svc.predict(X_test)
np.unique(pred_test == y_test, return_counts=True)

Aufgrund des ausgeglichenen Datensets ist die Accuracy ein gutes Maß für die Abstraktionsfähigkeit:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_test)

Das Ergebnis sieht fast genauso gut aus wie im Falle der BERT-Klassfikation.

Lass dir jetzt noch den Report ausgeben:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred_test))

Nicht immer muss also ein komplexes Modell tatsächlich so viel besser sein!

## Es muss nicht immer *Deep Learning* sein

Wie du gesehen hast, kann man auch einen herkömmlichen Klassifikator so trainieren, dass er eine ähnlich gute Performance wie ein wirklich komplex Sprachmodell erreicht.

Das ist selbstverständlich nicht für alle Aufgaben der Fall. In vorliegenden Fall klappt das gut, weil vermutlich einige *Buzzwords* darüber entscheiden, wie viele Kommentare ein Artikel erhält. Bei anderen Fragestellungen, in denen der Kontext eine größere Rolle spielt, können Transfer Learning Modelle sehr viel besser funktionieren als klassisches Machine Learning.